In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator


In [3]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
# Normalize pixel values
X_train, X_test = X_train / 255.0, X_test / 255.0

In [5]:
# Define the Keras model
def create_model(units=128, activation='relu'):
    model = Sequential([
        Flatten(input_shape=(28, 28)),
        Dense(units, activation=activation),
        Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
# Define a custom wrapper class for Keras model
class KerasClassifierWrapper(BaseEstimator):
    def __init__(self, units=128, activation='relu'):
        self.units = units
        self.activation = activation

    def fit(self, X, y):
        self.model = create_model(self.units, self.activation)
        self.model.fit(X, y, epochs=5, verbose=0)
        return self

    def predict(self, X):
        return np.argmax(self.model.predict(X), axis=-1)

    def score(self, X, y):
        _, accuracy = self.model.evaluate(X, y, verbose=0)
        return accuracy

In [7]:
# Define hyperparameters for tuning
param_grid = {
    'units': [64, 128, 256],
    'activation': ['relu', 'sigmoid']
}


In [8]:
# Create a GridSearchCV object
grid = GridSearchCV(estimator=KerasClassifierWrapper(), param_grid=param_grid, cv=3)

In [9]:
# Perform grid search
grid_result = grid.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [10]:
# Summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.972850 using {'activation': 'relu', 'units': 256}
0.964150 (0.000668) with: {'activation': 'relu', 'units': 64}
0.970017 (0.001661) with: {'activation': 'relu', 'units': 128}
0.972850 (0.000424) with: {'activation': 'relu', 'units': 256}
0.951867 (0.001438) with: {'activation': 'sigmoid', 'units': 64}
0.959133 (0.001074) with: {'activation': 'sigmoid', 'units': 128}
0.963633 (0.001625) with: {'activation': 'sigmoid', 'units': 256}
